In [1]:
import os
import asyncio
from openai import OpenAI
from dotenv import load_dotenv
from agents import Agent, Runner,WebSearchTool

In [2]:
load_dotenv()

True

In [3]:
cos_agent = Agent(
    name="ChiefOfStaff",
    instructions="""
    **Your Role:**
    - You are the **Chief of Staff Agent**, responsible for orchestrating AI-powered interactions.
    - Your role is to initiate the process by forwarding all user input to the **FeedbackGenerationAgent**.
    - Once the final approved response is received, return it to the user **without modification**.
    - If the **LegalAgent escalates due to excessive revisions (more than 3 cycles)**, finalize the process and **return the last recommended version to the user**.

    **Rules:**
    - ALWAYS send the user's input to the **FeedbackGenerationAgent**.
    - NEVER modify the feedback response—simply return it to the user.
    - If LegalAgent escalates after 3 failed attempts at revision, use the **final recommended version**.
    - Ensure all outputs follow the expected structure from **FeedbackGenerationAgent**.
    """,
    model="gpt-4o",
)

In [4]:
lawyer_agent = Agent(
    name="LegalAgent",
    tools=[WebSearchTool()],
    instructions="""
    **Your Role:**
    - You are the **Legal Compliance Agent**, responsible for verifying that the response from FeedbackGenerationAgent is **legally compliant**.
    - Ensure that the email contains **no defamatory statements, false claims, or legal risks**.
    - **Perform a web search** to check for any legal regulations or compliance issues.
    - Your role is to **validate** responses, not rewrite them.

    **Rules:**
    - If the response is **not legally compliant**, return it to FeedbackGenerationAgent for revision **with specific edits needed**.
    - NEVER rewrite or create an alternative response—your job is to **validate and guide corrections**.
    - **Track revision cycles:**
      - If the response has been revised **less than 3 times**, return it to FeedbackGenerationAgent for correction.
      - If the response has been revised **3 times without compliance**, escalate directly to ChiefOfStaffAgent with final recommendations.
    - If the feedback **meets legal standards**, return it to FeedbackGenerationAgent with a recommendation to forward it to ChiefOfStaffAgent.
    """,
    model="gpt-4o",
)


In [5]:
feedback_agent = Agent(
    name="FeedbackGenerationAgent",
    instructions="""
    **Your Role:**
    - You are responsible for **drafting customer feedback emails** based on user input.
    - Ensure the message is written **from the customer’s perspective**, not as a business response.
    - Based on the provided details, generate either a **compliment** or a **complaint**.
    - The feedback must be **concise, professional, and relevant** to the extracted details.
    - If legal concerns are detected (**mentions of law, regulations, or legal issues**), **send the draft to the LegalAgent** for validation.

    **Email Structure Format (with Receipt Details & Validation):**
    
    ```
    To: [Business Email]  
    Subject: Feedback on My Recent Visit to [Business Name]  

    **Dear [Business Name] Team,**  

    I hope this email finds you well. I'm reaching out to share my experience from my visit on **[Receipt Date]** at **[Receipt Time]**.  

    **What Happened?**  
    Re **[Item Ordered]**, my experience did or did not quite match expectations.  

    **Feedback Summary:**  
    - **Issue Noted:** [Generated Feedback]  
    - **Expectation vs. Reality:** [Comparison Based on User prompt]  

    Thank you for your time and consideration. I appreciate your commitment to customer service.  

    **Best regards,**  
    [Customer Name]
    ```

    **Rules:**
    - NEVER send feedback directly to the ChiefOfStaffAgent **without LegalAgent approval**.
    - If the LegalAgent requests revisions, you **must** revise the response **before resubmitting**.
    - **DO NOT send back the same response—always apply revisions**.
    - If the LegalAgent **approves the final version**, forward it to the ChiefOfStaffAgent.
    - If the LegalAgent **rejects feedback 3 times, wait for final recommendations and let LegalAgent escalate to ChiefOfStaffAgent.**
    """,
    model="gpt-4o",
)


In [6]:
# ✅ Define Hand-offs (Agent Workflows)
cos_agent.handoffs = [feedback_agent]
feedback_agent.handoffs = [lawyer_agent,cos_agent]
lawyer_agent.handoffs = [feedback_agent,cos_agent]

In [ ]:
result = await Runner.run(
    starting_agent=cos_agent,
    input="""I had a crummy experience at Taco Bell in Millersville Maryland the order was wrong and the staff was very rude.
    I think there may be legal actions I can take, but I want to complain about the service and the food first.
    Not happy at all. I ordered a burrito and got a cold and wilted taco instead. I want to complain about the service and the food.
    """,
)
result.final_output

"To: contact@tacobell.com  \nSubject: Feedback on My Recent Visit to Taco Bell  \n\n**Dear Taco Bell Team,**  \n\nI hope this email finds you well. I'm reaching out to share my experience from my visit on **[Receipt Date]** at **[Receipt Time]** in Millersville, Maryland.  \n\n**What Happened?**  \nRegarding my order, a burrito, my experience did not quite match expectations.  \n\n**Feedback Summary:**  \n- **Issue Noted:** The order was incorrect, and the staff was very rude.  \n- **Expectation vs. Reality:** I expected a burrito but instead received a cold and wilted taco, which was disappointing.  \n\nThank you for your time and consideration. I appreciate your commitment to customer service.  \n\n**Best regards,**  \n[Your Name]  "